Objectif:
Comprendre l'ensemble de données et le nettoyage (si nécessaire).
Créez des modèles de classification pour prédire si le type de cancer est malin ou bénin.
Ajustez également les hyperparamètres et comparez les mesures d'évaluation de divers algorithmes de classification.

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from sklearn.metrics import confusion_matrix

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
path = '/content/gdrive/My Drive/Breast_Cancer_Dataset'


In [ ]:
# Charger le dataset
data = pd.read_csv(path + '/breast-cancer.csv')


In [ ]:
# Explorer les données
#print(data.head())
#print(data.info())
#print(data.describe())


In [ ]:
#Id
data.drop('id', axis=1, inplace=True)  # Supprimer la colonne 'id'

In [ ]:
print(data['diagnosis'].unique())


In [ ]:
# Compter les occurrences de chaque valeur dans la colonne "diagnosis"
plt.figure(figsize=(6, 4))
counts = data['diagnosis'].value_counts()

# Créer un diagramme en barres
plt.bar(counts.index, counts.values, color=['red', 'green'])

# Ajouter des titres et des étiquettes d'axe
plt.title('Distribution des diagnostics')
plt.xlabel('Diagnostic')
plt.ylabel('Nombre de patients')
plt.legend(labels=['Benign', 'Malignant'])

# Afficher le diagramme
plt.show()

In [ ]:
# Séparer les variables indépendantes (X) et dépendantes (y)
X = data.drop('diagnosis', axis=1)
y = data['diagnosis']

In [ ]:
# Prétraiter les données
# Encoder les étiquettes en valeurs binaires (0 et 1)
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)
# Afficher les valeurs uniques des étiquettes encodées (pour vérifier qu'on a bien 0 et 1)
print("labels after encoding are: ", np.unique(y))#  'M': 0, 'B': 1
#data['diagnosis'] = data['diagnosis'].map({'M': 1, 'B': 0})  # Transformer la colonne 'diagnosis' en valeurs binaires

In [ ]:
# Remplacer les valeurs manquantes par la moyenne des valeurs de la colonne correspondante
data.fillna(data.mean(numeric_only=True), inplace=True)


In [ ]:
# Normaliser les données en utilisant MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

In [ ]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Normaliser les données
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(64, activation='relu', input_shape=(30,)),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

**Interpretation du Model**
Ce code définit un modèle de réseau neuronal séquentiel à l'aide de l'API Keras de TensorFlow. Le modèle comporte trois couches entièrement connectées, avec 64 neurones dans la couche d'entrée, 32 neurones dans la couche cachée et 1 neurone dans la couche de sortie. La fonction d'activation utilisée à la fois dans les couches d'entrée et cachée est ReLU (unité linéaire rectifiée), qui est couramment utilisée dans les réseaux de neurones.

La couche d'entrée a une forme d'entrée de (30,), ce qui signifie que le modèle attend des données d'entrée avec 30 entités. La couche de sortie a un seul neurone avec une fonction d'activation sigmoïde, ce qui est approprié pour les problèmes de classification binaire.

Le modèle comprend également une couche d'abandon avec un taux de 0,2, qui abandonne de manière aléatoire 20 % des unités d'entrée pendant l'entraînement pour aider à prévenir le surajustement.

In [ ]:
#from keras.layers.attention.multi_head_attention import activation
#model=Sequential()
#model.add(Dense(16,input_dim=30,activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(1))
#model.add(Activation('sigmoid'))



In [ ]:
# Compiler le modèle
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# Entraîner le modèle
history = model.fit(X_train, y_train,verbose=1, epochs=100,batch_size=64, validation_data=(X_test, y_test))

In [ ]:
"""!pip install flask
!pip install tensorflow-serving-api"""


In [ ]:
"""# Exporter le modèle
export_path = path 
tf.saved_model.save(model, export_path)"""

In [ ]:
#!tensorflow_model_server --port=8500 --rest_api_port=8501 --model_name=model --model_base_path=path



In [ ]:
# Évaluer la performance du modèle sur l'ensemble de test
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy: {:.2f}%'.format(test_acc*100))

In [ ]:
# Évaluer la performance du modèle sur l'ensemble de test
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(1,len(loss) + 1)
plt.plot(epochs,loss,'y',label='Training loss')
plt.plot(epochs,val_loss,'b',label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


acc=history.history['accuracy'] #Usage de Perf
val_acc=history.history['val_accuracy'] #Usage de Val
plt.plot(epochs,loss,'r',label='Training acc')
plt.plot(epochs,val_loss,'g',label='Validation acc')
plt.title('Training and Validation acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
y_pred=model.predict(X_test)
y_pred=(y_pred > 0.5)

#Matrice de Confusion
cm=confusion_matrix(y_test, y_pred)

sns.heatmap(cm, annot=True)

In [ ]:
# Enregistrer le modèle au format H5
#model.save('model.h5')

In [ ]:
import numpy as np
test_data1 = np.array([[13.08,15.71,85.63,520,0.1075,0.127,0.04568,0.0311,0.1967,0.06811,0.1852,0.7477,1.383,14.67,0.004097,0.01898,0.01698,0.00649,0.01678,0.002425,14.5,20.49,96.09,630.5,0.1312,0.2776,0.189,0.07283,0.3184,0.08183]])

#test_data=np.array[[13.08,15.71,85.63,520,0.1075,0.127,0.04568,0.0311,0.1967,0.06811,0.1852,0.7477,1.383,14.67,0.004097,0.01898,0.01698,0.00649,0.01678,0.002425,14.5,20.49,96.09,630.5,0.1312,0.2776,0.189,0.07283,0.3184,0.08183]]
test_data0 = np.array([[ 18.0, 100.0, 900.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7]])


In [ ]:
prediction1 = model.predict(test_data1)
prediction0 = model.predict(test_data0)

print(prediction1,prediction0)


Dans ce code, nous avons d'abord chargé le dataset et séparé les variables indépendantes (X) et dépendantes (y). Ensuite, nous avons normalisé les données à l'aide de la classe StandardScaler de scikit-learn.

Nous avons ensuite créé un modèle de réseau de neurones séquentiel avec trois couches : une couche d'entrée avec 64 neurones (pour correspondre au nombre de variables indépendantes), une couche cachée avec 32 neurones et une fonction d'activation ReLU, et une couche de sortie avec une fonction d'activation sigmoïde pour prédire la probabilité de cancer malin. Nous avons compilé le modèle avec une fonction de perte binary_crossentropy et une métrique d'accuracy.

Nous avons ensuite entraîné le modèle pendant 100 époques sur l'ensemble d'entraînement, en utilisant l'ensemble de test comme ensemble de validation. Enfin, nous avons évalué la performance du modèle sur l'ensemble de test à l'aide de la méthode evaluate.

In [ ]:
"""%%writefile app.py
"""

In [ ]:
"""!export FLASK_APP=app.py
!flask run --port 5000"""